In [1]:
using LinearAlgebra

In [2]:
include("C:/Users/mjski/Desktop/Active-Work/git-projects/scripts/scripts/spacecraft_adcs.jl")

p2DCM (generic function with 1 method)

In [24]:
function ECEF_LVLH(R, V)
    x̂ = R ./ norm(R)
    ŷ = V ./ norm(V)
    ẑ = cross(x̂, ŷ) ./ norm(cross(x̂, ŷ))
    
    return x̂, ŷ, ẑ
end

function C_LVLH_ECEF(Rx, Ry, Rz, Vx, Vy, Vz)
    #x, y, z = R
    x̂l, ŷl, ẑl = ECEF_LVLH([Rx, Ry, Rz], [Vx, Vy, Vz])
    
    R̂ = [1.0, 0.0, 0.0]
    
    ϕ = acos(dot(R̂, x̂l))
    
    if ϕ != 0.0
        α = cross(R̂, x̂l) ./ norm(cross(R̂, x̂l))
        return cos(ϕ) .+ (1 - cos(ϕ)) .* α * α' .- sin(ϕ) .* cross_mat(α)
    else
        return I(3)
    end
end

function ecef2geocentric_latlon(x, y, z)
    λ = atan(y, x)
    
    r = sqrt(x^2 + y^2 + z^2)
    p = sqrt(x^2 + y^2)
    
    ϕc = atan(p, z)

    return π/2 - ϕc, λ 
end

ecef2geocentric_latlon (generic function with 1 method)

In [4]:
R_mag = 6378 + 250
θ = π/6
R = [cos(θ) * R_mag, sin(θ) * R_mag, 0]

V_mag = sqrt(398600 / R_mag)
V = [cos(π/2 + θ) * V_mag, sin(π/2 + θ) * V_mag, 0]

3-element Vector{Float64}:
 -3.8774606725730463
  6.715958889246711
  0.0

In [13]:
x, y, z = ECEF_LVLH(R, V)

([0.8660254037844387, 0.49999999999999994, 0.0], [-0.49999999999999983, 0.8660254037844388, 0.0], [0.0, -0.0, 1.0])

In [14]:
2*x

3-element Vector{Float64}:
 1.7320508075688774
 0.9999999999999999
 0.0

In [15]:
2*y

3-element Vector{Float64}:
 -0.9999999999999997
  1.7320508075688776
  0.0

In [12]:
4 * [R ./ 6378]

1-element Vector{Vector{Float64}}:
 [3.599884839312173, 2.0783944810285355, 0.0]

In [17]:
4 * R_mag / 6378

4.156788962057071

In [18]:
rad2deg(π/6)

29.999999999999996

In [23]:
R

3-element Vector{Float64}:
 5740.01637628326
 3313.9999999999995
    0.0

In [22]:
C_LVLH_ECEF(R[1], R[2], R[3], V[1], V[2], V[3]) * [1,0,0]

3-element Vector{Float64}:
 0.8660254037844387
 0.3660254037844389
 0.8660254037844387

In [ ]:
function C_LVLH_ECEF2(R)
    x, y, z = R
    δ, λ = ecef2geocentric_latlon(x, y, z)
    
    return par_2(λ)
end